Creating JSON file for our dataset

In [1]:
import json

#  Define input & output file paths
input_file = r"C:\Users\gvais\OneDrive\Desktop\faq\faq text.txt"
output_file = r"C:\Users\gvais\OneDrive\Desktop\faq\faq_data.json"

In [2]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\gvais\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import chardet  # Detects file encoding

In [4]:
# List to store extracted FAQs
faqs = []
question = None
answer = ""

In [5]:
# Detect File Encoding First (To Avoid Unicode Errors)
with open(input_file, "rb") as f:
    raw_data = f.read(50000)  # Read a portion of the file
    result = chardet.detect(raw_data)
    detected_encoding = result["encoding"]

print(f" Detected Encoding: {detected_encoding}")

 Detected Encoding: Windows-1252


In [6]:
#  Read the text file with detected encoding (Avoid Unicode Errors)
with open(input_file, "r", encoding=detected_encoding, errors="ignore") as f:
    lines = f.readlines()

In [7]:
#  Try multiple encodings (utf-8-sig, latin-1, or errors="replace" to avoid crashes)
with open(input_file, "r", encoding="utf-8-sig", errors="replace") as file:
    for line in file:
        line = line.strip()

        if line.startswith("Q-"):  # Identify questions
            if question and answer:  # Store previous Q&A before starting a new one
                faqs.append({"question": question, "answer": answer.strip()})
            question = line[2:].strip()  # Remove 'Q-' and store question
            answer = ""  # Reset answer for new question

        elif line.startswith("A-"):  # Identify answers
            answer = line[2:].strip()  # Remove 'A-' and store answer

        else:  # Append multi-line answers
            answer += " " + line



In [8]:

# Append the last Q&A pair
if question and answer:
    faqs.append({"question": question, "answer": answer.strip()})

# Save FAQs to JSON file
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump({"faqs": faqs}, json_file, indent=4, ensure_ascii=False)

print(f" FAQs successfully extracted and saved to: {output_file}")

 FAQs successfully extracted and saved to: C:\Users\gvais\OneDrive\Desktop\faq\faq_data.json


In [9]:
import json

faq_file = r"C:\Users\gvais\OneDrive\Desktop\faq\faq_data.json"  # Update path

In [10]:
#  Load JSON file correctly
with open(faq_file, "r", encoding="utf-8") as f:
    faqs = json.load(f)

In [11]:
#  Check the type of `faqs`
print("Type of faqs:", type(faqs))  # Should be <class 'list'>

#  If it's a dictionary, print keys
if isinstance(faqs, dict):
    print("Dictionary keys:", faqs.keys())

#  If it's a list, print first 3 entries
if isinstance(faqs, list):
    print(faqs[:3])  # Should show a list of dictionaries

Type of faqs: <class 'dict'>
Dictionary keys: dict_keys(['faqs'])


Tokenizing our data

In [12]:
#loading llama's pretrained tokenizer
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

text = "What is BharatPe? BharatPe is a financial technology company..."
tokens = tokenizer(text, return_tensors="pt")

print(tokens.input_ids)  # Tokenized representation


C:\Users\gvais\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[    1,  1724,   338,   350,  8222,   271, 15666, 29973,   350,  8222,
           271, 15666,   338,   263, 18161, 15483,  5001,   856]])


In [13]:

# Load your FAQs
with open(r"C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_faqs.json", "r", encoding="utf-8") as f:
    faqs = json.load(f)

In [14]:
import transformers
print(transformers.__version__)

4.48.1


In [15]:
#  Load JSON file correctly
with open(faq_file, "r", encoding="utf-8") as f:
    faqs_dict = json.load(f)  # This loads a dictionary

In [16]:
#  Extract the FAQ list from the dictionary
faqs = faqs_dict.get("faqs", [])  # Safely get the list

#  Check if it's now a list
print("Type of faqs after extraction:", type(faqs))  # Should be <class 'list'>
print(faqs[:3])  # Print first 3 FAQs to verify

Type of faqs after extraction: <class 'list'>
[{'question': 'what is bharatpe?', 'answer': "bharatpe is a new way to pay, and makes spending easy. in collaboration with lending partners, you can spend now and pay next month. a line of credit is offered through bharatpe by rbi registered lenders (after necessary checks and kyc completion) which can be used anytime, anywhere to purchase goods and services - either by scanning a qr, sending money directly to your bank account, making utility bill payments or availing a personal loan from the assigned bharatpe limit. when it's time to pay your bill, you can clear your dues in one go, or convert to an equated monthly installment (emi) with tenure of your choice. you can also earn exciting cashback and rewards. get started now!"}, {'question': 'how do i sign up for bharatpe?', 'answer': 'you can sign up for bharatpe by verifying your mobile number and then completing your kyc using pan and aadhaar number or any other information as required 

In [17]:


# Tokenize all questions and answers
tokenized_faqs = []
for faq in faqs:
    tokens = tokenizer(faq["question"] + " " + faq["answer"], return_tensors="pt")
    tokenized_faqs.append(tokens.input_ids.tolist())


In [18]:
# Save the tokenized dataset
with open("tokenized_faqs.json", "w") as f:
    json.dump(tokenized_faqs, f)


VECTOR EMBEDDING

In [19]:
import json
import torch
from transformers import AutoTokenizer, AutoModel

In [20]:
#  Load the pre-trained embedding model
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Change to OpenAI model if needed
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [21]:
#  Load BPE Tokenized Dataset
file_path = r"C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_faqs.json"  # Use full path
with open(file_path, "r", encoding="utf-8") as file:
    faq_data = json.load(file)

In [22]:
# Function to Convert Text to Vector Embeddings
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

In [23]:
#  Generate Embeddings for Each FAQ
embeddings = []
for item in faq_data["faqs"]:
    vector = generate_embedding(item["question"] + " " + item["answer"])
    embeddings.append({"question": item["question"], "embedding": vector})


In [24]:
#  Save Embeddings to a JSON File
embedding_file = r"C:\Users\gvais\OneDrive\Desktop\faq\faq_embeddings.json"
with open(embedding_file, "w", encoding="utf-8") as f:
    json.dump(embeddings, f, indent=4)

print(f" Embeddings saved at: {embedding_file}")

 Embeddings saved at: C:\Users\gvais\OneDrive\Desktop\faq\faq_embeddings.json


Generate & Store Embeddings using Ollama

In [25]:
import json
import ollama
import chromadb

In [26]:
#  Load the FAQ dataset
file_path = r"C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_faqs.json"
with open(file_path, "r", encoding="utf-8") as file:
    faq_data = json.load(file)

In [27]:
#  Initialize ChromaDB for Storing Embeddings
chroma_client = chromadb.PersistentClient(path=r"C:\Users\gvais\OneDrive\Desktop\faq\chroma_db")
collection = chroma_client.get_or_create_collection(name="faq_embeddings")

In [28]:
# ran for 1+ hr and still running so commemted it out for now
# #  Generate and Store Embeddings
# for item in faq_data["faqs"]:
#     text = item["question"] + " " + item["answer"]
#     response = ollama.embeddings(model="mistral", prompt=text)  # Use your preferred LLaMA model
#     embedding = response["embedding"]

#     # Store in ChromaDB
#     collection.add(
#         ids=[item["question"]],  # Use question as unique ID
#         embeddings=[embedding],
#         metadatas=[{"question": item["question"], "answer": item["answer"]}]
#     )

# print(" Embeddings successfully generated and stored in ChromaDB!")


Convert Our Tokenized Dataset into QLoRA-Compatible JSONL

In [29]:

#  Paths for input and output files
bpe_tokenized_file = r"C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_faqs.json"  # BPE tokenized dataset
output_jsonl_file = r"C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_qlora.jsonl"  # QLoRA fine-tuning dataset

In [69]:
#  Load the tokenized FAQ dataset
with open(bpe_tokenized_file, "r", encoding="utf-8") as file:
    faq_data = json.load(file)


In [70]:
#  Convert the dataset to QLoRA format
with open(output_jsonl_file, "w", encoding="utf-8") as jsonl_file:
    for item in faq_data["faqs"]:
        jsonl_entry = {
            "instruction": item["question"],  # Question from FAQ dataset
            "input": "",  # No additional input for now
            "output": item["answer"]  # Answer from FAQ dataset
        }
        jsonl_file.write(json.dumps(jsonl_entry) + "\n")  # Write in JSONL format

print(f" QLoRA dataset successfully saved at: {output_jsonl_file}")

 QLoRA dataset successfully saved at: C:\Users\gvais\OneDrive\Desktop\faq\bharatpe_qlora.jsonl


 Split into Training & Validation Sets

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
#  Load formatted dataset
with open(output_file, "r", encoding="utf-8") as f:
    data = f.readlines()

In [32]:
#  Split into 90% training & 10% validation
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

In [33]:
#  Save splits
with open("train.jsonl", "w", encoding="utf-8") as f:
    f.writelines(train_data)

with open("val.jsonl", "w", encoding="utf-8") as f:
    f.writelines(val_data)

print(" Train-Test split done!")


 Train-Test split done!


Load the Model with LoRA (QLoRA for Efficient Fine-Tuning)

In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import torch

In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [38]:
#  Load tokenizer & model
model_name = "facebook/opt-1.3b"  # You can also use "mistralai/Mistral-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [73]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

In [74]:
# Use a smaller Facebook model (OPT-1.3B instead of LLaMA-7B)
model_name = "facebook/opt-1.3b"

In [75]:
# Enable 4-bit quantization for lower memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [76]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

In [77]:

# Load Model with CPU offloading (for low memory systems)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    max_memory={0: "5GB", "cpu": "2GB"}  # Adjust memory allocation
)

print("Model loaded successfully!")


Model loaded successfully!


In [78]:


# # Define model name (Change this to another LLM if needed)
# model_name = "meta-llama/Llama-2-7b-chat-hf"  # Example model, change as needed

# # Load model with 4-bit quantization (QLoRA)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True,
# )

# # Load Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token if needed

# # Load Model with QLoRA
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# print("Model loaded successfully!")


In [79]:
with open(r"C:\Users\gvais\OneDrive\Desktop\faq\faq_data.json", 'r') as f:
    data = f.read()
    print(len(data))

78139


In [80]:
with open('train.jsonl', 'r') as f:
    data = f.read()
    print(len(data))

7


In [81]:
#  Define training parameters
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
)


C:\Users\gvais\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing and Generating response

In [82]:
# Define a test prompt
prompt = "What is UPI and how does it work?"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
output = model.generate(**inputs, max_length=100)

# Decode and print response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Model Response:", response)

Model Response: What is UPI and how does it work?
<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>


In [84]:
#loading dataset for finetuning
from datasets import load_dataset

# Load JSONL dataset
dataset = load_dataset("json", data_files="train.jsonl")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]Failed to load JSON from file 'C:\Users\gvais\OneDrive\Desktop\BharatPe Chatbot\BharatPe-copilot\train.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to string in row 0
Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset